In [1]:
import pandas as pd
import numpy as np
from model_train.base.data import process_data
from model_train.base import model
import pickle
from sklearn.model_selection import train_test_split


In [45]:
#import data from data
df = pd.read_csv('data/censusclean.csv')

In [50]:
df[df['salary'] == ' >50K']

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
10,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K
11,30,State-gov,141297,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32539,71,?,287372,Doctorate,16,Married-civ-spouse,?,Husband,White,Male,0,0,10,United-States,>50K
32545,39,Local-gov,111499,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,20,United-States,>50K
32554,53,Private,321865,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K


In [49]:
df['salary'].value_counts()

 <=50K    24720
 >50K      7841
Name: salary, dtype: int64

In [46]:
df.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
train, test = train_test_split(df, test_size=0.20)

In [5]:
cat_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]
X_train, y_train, encoder, lb = process_data(
    train, categorical_features=cat_features, label="salary", training=True
)

In [6]:
X_test, y_test, _, _ = process_data(test, categorical_features=cat_features, label="salary", training=False, encoder=encoder, lb=lb)


In [7]:
output_model = model.train_model(X_train, y_train)

In [8]:
train_preds = model.inference(output_model, X_train)
test_preds = model.inference(output_model, X_test)

In [9]:
precision, recall, fbeta = model.compute_model_metrics(y_test,test_preds)
print (f"prec:{precision}, recall:{recall}, fbeta:{fbeta}")

prec:0.7794486215538847, recall:0.5984605516356639, fbeta:0.6770682148040638


In [10]:
model.compute_slice_metrics(test,cat_features,output_model,encoder,lb)

In [8]:
from fastapi.encoders import jsonable_encoder
from model_train.base import model
from model_train.base.data import process_data
import pandas as pd
import pickle

In [64]:
  example = {"age": 31,
    "workclass": "Private",
    "fnlgt": 45781,
    "education": "Masters",
    "education-num": 14,
    "marital-status": "Never-married",
    "occupation": "Prof-specialty",
    "relationship": "Not-in-family",
    "race": "White",
    "sex": "Female",
    "capital-gain": 14084,
    "capital-loss": 0,
    "hours-per-week": 50,
    "native-country": "United-States"}

In [58]:
    input = jsonable_encoder(example)
    df = pd.DataFrame(input, index=[0])

In [66]:
df = pd.DataFrame(example, index=[0])

In [60]:
    with open('model/model.pkl', 'rb') as f:
        output_model = pickle.load(f)
    with open('model/encoder.pkl', 'rb') as f:
        encoder = pickle.load(f)
    with open('model/lb.pkl', 'rb') as f:
        lb = pickle.load(f)

In [61]:
    cat_features = [
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country"]

In [62]:
    X_test, y_test, _, _ = process_data(df, categorical_features=cat_features, label=None, training=False, encoder=encoder, lb=lb)

  

In [63]:
 model.inference(output_model, X_test)

array([1])